### pyspark read kafka topic

In [94]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_data"
LIMIT = 1

In [95]:
BOOTSTRAP_SERVERS = ['datafabric-kafka-kafka-0.datafabric-kafka-kafka-brokers.kafka-farm.svc.cluster.local:9092', 'datafabric-kafka-kafka-1.datafabric-kafka-kafka-brokers.kafka-farm.svc.cluster.local:9092','datafabric-kafka-kafka-2.datafabric-kafka-kafka-brokers.kafka-farm.svc.cluster.local:9092']
TOPICS = 'datafabric'
CONSUMER_GROUP = 'test-datafabric'

In [96]:
from pydatafabric.ye import get_spark
from pydatafabric.gcp import bq_to_df

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/spark-sql-kafka-0-10_2.12-3.1.2.jar")

In [97]:
spark.conf.get('spark.jars')

'hdfs:///jars/spark-bigquery-with-dependencies_2.12-latest.jar,gs://emart-datafabric-resources/jars/spark-sql-kafka-0-10_2.12-3.1.2.jar'

In [89]:
df = bq_to_df(f"""
    select review_id, comments
    from  `{PROJECT}.{DATASET}.{TABLE}`
    limit {LIMIT}
""", spark_session=spark)

In [90]:
df.show(10)

+----------------+--------------------------------+
|       review_id|                        comments|
+----------------+--------------------------------+
|2208051914367759|싱싱은 한데
아무 맛도 안나요...|
+----------------+--------------------------------+



In [91]:
df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- comments: string (nullable = true)



### spark from kafaka topic read

In [64]:
# !pip install kafka-python

In [62]:
# from kafka import KafkaProducer
# from json import dumps
# import time

# producer = KafkaProducer(acks=0,
#                          compression_type='gzip',
#                          bootstrap_servers=BOOTSTRAP_SERVERS,
#                          value_serializer=lambda x: dumps(x).encode('utf-8')
#                          )

# start = time.time()

# for i in range(10000):
#     data = {'str': 'result' + str(i)}
#     producer.send('test', value=data)
#     producer.flush()

# print("elapsed :", time.time() - start)

In [63]:
# import json
# from kafka import KafkaConsumer

# consumer = KafkaConsumer(
#      TOPICS,
#      bootstrap_servers=BOOTSTRAP_SERVERS,
#      enable_auto_commit=True,
#      group_id="test1-group",
#      value_deserializer=lambda x: loads(x.decode('utf-8')),
#      consumer_timeout_ms=1000
# )

# # # consumer 리스트를 가져온다
# # print('[begin] get consumer list')
# for message in consumer:
#     print("Topic: %s, Partition: %d, Offset: %d, Key: %s, Value: %s" % (
#         message.topic, message.partition, message.offset, message.key, message.value
#     ))
# # print('[end] get consumer list')


In [93]:
df = spark.read \
  .format("kafka") \
  .option("subscribe", TOPICS) \
  .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
  .option("failOnDataLoss", "false") \
  .option("subscribe", "topic1") \
  .option("includeHeaders", "true") \
  .option("startingOffsets", "latest") \
  .load()

# df.write \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
#   .option("topic", TOPICS) \
#   .save()

# if starting_offsets is not None:
#     print(starting_offsets)
#     df_reader = df_reader \
#         .option("startingOffsets", starting_offsets)\
#         .option("auto.offset.reset", "earliest") \
#         .option("failOnDataLoss", "false")
    
# if CONSUMER_CONFIGS != '':
#     config_tokens = CONSUMER_CONFIGS.split(',')
#     print(config_tokens)
    
#     for token in config_tokens:
#         key_value = token.split('=')
#         print(key_value)
        
#         df_reader = df_reader.option(f"kafka.{key_value[0]}", key_value[1])
        
# df = df_reader.load()
# df.printSchema()
# df.show()

Py4JJavaError: An error occurred while calling o1625.load.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.KafkaSourceProvider$
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateBatchOptions(KafkaSourceProvider.scala:336)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.createRelation(KafkaSourceProvider.scala:127)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:355)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:325)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$3(DataFrameReader.scala:307)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:307)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:225)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMe

In [81]:
spark.stop()